In [ ]:
import os
from zlm import AutoApplyModel
from creds import OPENAI_KEY

#os.environ['OPENAI_KEY'] = OPENAI_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


job_llm = AutoApplyModel(
    api_key=None,
    provider="GPT", #ENTER PROVIDER <gemini> or <openai>
    downloads_dir=r"/Users/aaronrdankert/projects/job_hunter/output_files", #[optional] ENTER FOLDER PATH WHERE FILE GET DOWNLOADED, By default, 'downloads' folder
    model="chatgpt-4o-latest"
)


url = r"https://careers-jobyaviation.icims.com/jobs/3483/senior-backend-software-engineer%2c-factory-systems/job"
resume_pdf_path = r"input_data/Grad School Resume.pdf"
job_llm.resume_cv_pipeline(
    job_url=url,
    user_data_path= resume_pdf_path # .pdf or .json
) # Return and downloads curated resume and cover letter.

Markdown 

In [5]:
from creds import OPENAI_KEY
from zlm import AutoApplyModel

job_llm = AutoApplyModel(
    api_key=OPENAI_KEY,  # Pass the actual API key value
    provider="GPT",  # ENTER PROVIDER <gemini> or <openai>
    downloads_dir=r"/Users/aaronrdankert/projects/job_hunter/output_files",  # [optional] ENTER FOLDER PATH WHERE FILE GET DOWNLOADED, By default, 'downloads' folder
    model="chatgpt-4o-latest"
)

url = r"https://careers-jobyaviation.icims.com/jobs/3483/senior-backend-software-engineer%2c-factory-systems/job"
resume_pdf_path = r"input_data/Grad School Resume.pdf"
job_llm.resume_cv_pipeline(
    job_url=url,
    user_data_path=resume_pdf_path  # .pdf or .json
)

Starting Auto Resume and CV Pipeline

Fetching user data...


2024-11-04 21:28:57.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.354 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:28:57.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Function user_data_extraction took 12.8141 seconds to execute

Extracting job details...
It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.
Unable to web scrape the job description.
Function job_details_extraction took 0.0071 seconds to execute

Generating Resume Details...


2024-11-04 21:29:18.690 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:29:18.691 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:29:18.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:29:18.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:29:18.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-04 21:29:18.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


'NoneType' object is not subscriptable
cannot access local variable 'resume_path' where it is not associated with a value
